In [1]:
import os,glob
import os
import xml.etree.ElementTree as ET
from datetime import datetime

In [2]:
cc=os.getcwd()
path = "Data"+os.sep+"training"+os.sep+"Raw Data"+os.sep

In [3]:
errors_list=[]

In [4]:
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    
    txt_filename = os.path.splitext(xml_file)[0] + '.txt'
    with open(txt_filename, 'w') as out_file:
        for obj in root.iter('object'):
            difficult = obj.find('difficult').text
            if int(difficult) == 1:
                continue
            cls = obj.find('name').text
            if cls not in classes:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
                float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((w, h), b)
            out_file.write(f"{cls_id} {' '.join(map(str, bb))}\n")

# Define your classes here in the specific order
classes = [
    "ALLIGATOR [H] (sq.m)", "BLOCK (sq.m)", "LONGITUDINAL (m)", "TRANSVERSE (m)", 
    "RUTTING (sq.m)", "RAVELING [H] (sq.m)", "CORRUGATION (sq.m)", "POTHOLE (count)", 
    "DEPRESSION (sq.m)", "EDGE (m)", "RAIL ROAD CROSSING (sq.m)", "BLEEDING (sq.m)", 
    "JOINT REFLECTION (m)", "PATCHING (sq.m)", "POLISHED AGGREGATE (sq.m)", "SHOVING (sq.m)", 
    "SLIPPAGE (sq.m)", "BUMPS & SAGS (m)", "SWELL (m)", "WEATHERING (sq.m)", "CARRIAGEWAY", 
    "ALLIGATOR [M] (sq.m)", "ALLIGATOR [L] (sq.m)", "RAVELING [M] (sq.m)"
]

# Directory to search for XML files
input_dir = path

# Walk through the directory and convert all XML files
for root, _, files in os.walk(input_dir):
    for file in files:
        if file.endswith('.xml'):
            errors_list.append(f"XML File: {file}")
            xml_file = os.path.join(root, file)
            convert_annotation(xml_file)
            os.remove(xml_file)

print("XML Conversion completed.")


XML Conversion completed.


In [5]:
def find_data(path):
    file_paths = []
    for root, directories, files in os.walk(path):
        for filename in files:
            file_path = os.path.join(root, filename)
            file_paths.append(file_path)
    return file_paths

In [6]:
def check_images_format(data_dir):
    global errors_list
    imglist = glob.glob(os.path.join(data_dir, '**', '*.jpg'), recursive=True) + glob.glob(os.path.join(data_dir, '**', '*.png'), recursive=True)
    print(imglist)
    for img in imglist:
        if img.endswith('.png'):
            errors_list.append(f'Image is in png format: {img}')
            dir=os.path.dirname(img)
            old_imgname=os.path.basename(img).split(".")[0]
            new_imgname=old_imgname+'.jpg'
            os.rename(img,dir+os.sep+new_imgname)
    print("Images Format checked!")
            

In [7]:
def check_classes(data_dir):
    """Checks if the given labels are valid for the current class. 
    Args:
    lables (list): A list of labels to check.
    Returns:
    bool: True if all labels are valid, False otherwise.
    """
    
    txt_list = glob.glob(os.path.join(data_dir, '**', '*.txt'), recursive=True)
    txt_list = [x for x in txt_list if 'classes.txt'  not in x]
    
    global errors_list
    
    for ttxt in txt_list:
        # print(ttxt)
        deleteee=False
        dir=os.path.dirname(ttxt)
        imgfile=ttxt.split(os.sep)[-1].split('.')[0]
        with open(ttxt,'r') as f:
            lines = f.readlines()
            if len(lines) == 0:
                errors_list.append(f'Empty label file : {ttxt}')
                deleteee = True
                
            for line in lines:
                if len(line.split(' ')) != 5:
                    errors_list.append(f'Empty line in: {ttxt}')
                    lines.remove(line)
                elif int(line.split(' ')[0]) > 23:
                    errors_list.append(f"Unknown class {line.split(' ')[0]} in : {ttxt}")
                    lines.remove(line)
                else:
                    pass
                
        if deleteee == True:
            os.remove(ttxt)
            os.remove(dir+os.sep+imgfile+'.jpg')
        
        else:    
            with open(ttxt,'w') as file:
                file.writelines(lines)
    print("Classes checked!")      


    

In [8]:
def check_for_models():
    pass

In [9]:
# conveting png to jpg
#deleting unannotated files and labels with no images or images/labels with different names

def check_annotations(data_dir):
    
    imgs_list = glob.glob(os.path.join(data_dir, '**', '*.jpg'), recursive=True)
    imgnamelsit=[i.split(os.sep)[-1].split('.')[0] for i in imgs_list]
    no_of_img=len(imgnamelsit)
    extra_pics=0
    
    txt_list = glob.glob(os.path.join(data_dir, '**', '*.txt'), recursive=True)
    txt_list = [x for x in txt_list if 'classes.txt'  not in x]
    txtnamelsit=[i.split(os.sep)[-1].split('.')[0] for i in txt_list]
    
    for img in imgs_list:
        imgname=img.split(os.sep)[-1].split('.')[0]
        if imgname not in txtnamelsit:
            extra_pics+=1
            errors_list.append(f'Image with no label: {img}')
            os.remove(img)
    
            
    imgs_list = glob.glob(os.path.join(data_dir, '**', '*.jpg'), recursive=True)
    imgnamelsit=[i.split(os.sep)[-1].split('.')[0] for i in imgs_list]
    
    txt_list = glob.glob(os.path.join(data_dir, '**', '*.txt'), recursive=True)
    txt_list = [x for x in txt_list if 'classes.txt'  not in x]
    txtnamelsit=[i.split(os.sep)[-1].split('.')[0] for i in txt_list]
    no_of_lbl=len(txtnamelsit)
    extra_lbl=0
    
    for txt in txt_list:
        txtname=txt.split(os.sep)[-1].split('.')[0]
        if txtname not in imgnamelsit:
            extra_lbl +=1
            errors_list.append(f'Label with no image: {txt}')  
            os.remove(txt)
            
    print(f'Total Images: {no_of_img} Total Labels: {no_of_lbl}') 
    print(f'Extra Images: {extra_pics} Extra Labels: {extra_lbl}') 
    print(f'Labels without extra: {no_of_lbl-extra_lbl}')


def remove_extra_files(path):
    for root, _, files in os.walk(path):
        for file in files:
            # if file.endswith('.xml') or file.endswith('.SRT') or file.endswith('.xlsx'):
            if file.endswith('.xml') or file.endswith('.SRT') or file.endswith('.xlsx') or file.endswith('classes.txt'):
                file_path = os.path.join(root, file)
                os.remove(file_path)

In [10]:
check_images_format(path)
check_classes(path)
check_annotations(path)
remove_extra_files(path)

['Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h11m21s548.jpg', 'Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h11m27s830.jpg', 'Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h11m33s141.jpg', 'Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h11m44s127.jpg', 'Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h11m46s462.jpg', 'Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h11m48s509.jpg', 'Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h11m51s695.jpg', 'Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h11m53s451.jpg', 'Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h11m58s083.jpg', 'Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h12m00s840.jpg', 'Data\\training\\Raw Data\\1. Nazar Road\\Segment 01\\vlcsnap-2024-10-18-00h12m

In [11]:
now = datetime.now()
current_time_str = now.strftime("%Y-%m-%d_%H-%M-%S")
errors_file_name = f"Data/Errors_({current_time_str}).txt"

with open(errors_file_name, "w") as file:
    for item in errors_list:
        file.write(f"{item}\n")

print(f"Errors saved to {errors_file_name}")

Errors saved to Data/Errors_(2024-10-25_17-36-25).txt
